In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.10.254', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select count(*) from store.usage_basic_kpi_fact_v1_p_{} where date='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2019, 10, 31)
    start = datetime.date(2017, 9, 20)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(granularity):
    date_list = {}
    if granularity == 'daily':
        collect_date = get_daily_date_list()
    for x in collect_date:
        if date_list.has_key(x[0][:7]):
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda x: datetime.datetime.strptime(x[0] + str(-01), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(m, day))
            db_count = result[0][0]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity={}/date={}/'
            v3_count = spark.read.format("delta").load(v3_path.format(graularity, day)).count()

            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print "Completeness Test Pass! date : {}".format(day)


graularity_list = ["daily"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF
select device_id,store_id,date,app_id,kpi,estimate from plproxy.execute_select_nestloop(\$proxy\$ 
    select device_id,store_id,date,app_id,kpi,estimate
    from mu.app_daily
    where 
        date = '2019-11-01'
\$proxy\$) t (device_id SMALLINT,store_id INT,date DATE,app_id BIGINT,kpi SMALLINT, estimate FLOAT) limit 50;
EOF

In [0]:

df = spark.read.parquet('s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=v3.0.0/range_type=DAY/date=2019-11-30/')
# df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date=2019-10-31/")
print df.filter('AU is not null').count()

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF
select count(app_id) from plproxy.execute_select_nestloop(\$proxy\$ 
    select app_id
    from mu.app_daily
    where 
        date = '2019-11-30' and kpi=1
\$proxy\$) t (app_id BIGINT);
EOF

In [0]:

df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/month=2019-10/")
print df.filter("date='2019-10-31'").count()
print df.filter("date='2019-10-31' and kpi=1").count()

In [0]:

v1_df_weekly = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=weekly/date=2015-05-23/')
print 'weekly', v1_df_weekly.filter("device_code like 'and%' and (est_install_penetration!=0 or est_installs!=0 or est_open_rate!=0 or est_installs_country_share!=0)").select('country_code').distinct().count()

In [0]:

v1_df_weekly = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=weekly/date=2015-06-13/')
print 'weekly', v1_df_weekly.filter("device_code like 'and%' and est_install_penetration!=0 and est_installs!=0 and est_open_rate!=0 and est_installs_country_share!=0").select('country_code').distinct().count()

In [0]:

df = spark.read.parquet('s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=v3.0.0/range_type=DAY/date=2019-12-01/')
# df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date=2019-10-31/")
print df.filter('AU is not null').count()

In [0]:
%%sh
aws s3 ls s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=DAY/


In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-usage/unified/usage.legacy-mu_app.v1/fact/granularity=daily/

In [0]:

df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=daily/date=2019-01-29/')
df.select('app_id', 'device_code', 'country_code', 'est_average_active_users', 'est_share_of_users').filter("app_id=326251330 and device_code='ios-all' and (country_code='CN' or country_code='WW')").distinct().show()
df2 = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v1/fact/granularity=daily/date=2019-01-29/')
df2.select('app_id', 'device_code', 'country_code', 'est_average_active_users', 'est_average_active_users_country_share').filter("app_id=326251330 and device_code='ios-phone' and (country_code='CN' or country_code='WW')").show()

In [0]:

df = spark.read.parquet('s3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/granularity=daily/date=2019-01-29/')
df.select('app_id', 'device_code', 'country_code', 'est_average_active_users', 'est_share_of_users').filter("device_code='android-phone' and app_id=20600001429645 and country_code in ('CN', 'WW')").distinct().show()

In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select count(*) from store.usage_basic_kpi_fact_v6_p_{} where date='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2019, 12, 31)
    start = datetime.date(2018, 4, 1)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(m, day))
            db_count = result[0][0]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/' \
                      'granularity={}/date={}/'
            v3_count = spark.read.format("delta").load(v3_path.format(graularity, day)).count()

            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print "Completeness Test Pass! date : {}".format(day)
            test_result.append((graularity, day, v3_count, db_count))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'v3_count', 'db_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_v3_db_count_0616/daily/",
            mode="append",
            partitionBy=["type"])

    retry(write_test_result, (df_write_result,), {}, interval=10)


graularity_list = ["daily"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:

df =spark.read.format("delta").load('s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_v3_db_count_0616/weekly/').orderBy('date').collect()
for row in df:
    print row['date'],'\t',int(row['v3_count']),'\t',row['db_count']

In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select count(*) from usage.usage_basic_kpi_fact_v6_p_{} where date='{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 5, 23)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(m, day, graularity))
            db_count = result[0][0]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/' \
                      'granularity={}/date={}/'
            v3_count = spark.read.format("delta").load(v3_path.format(graularity, day)).count()

            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print "Completeness Test Pass! date : {}".format(day)
            test_result.append((graularity, day, v3_count, db_count))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'v3_count', 'db_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_v3_db_count_0616/daily/",
            mode="overwrite",
            partitionBy=["type"])

    retry(write_test_result, (df_write_result,), {}, interval=10)


graularity_list = ["daily"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=usage;
select * from usage_basic_kpi_fact_v6 where granularity='monthly' and date='2020-05-31' and app_id=100 and country_code='US';
EOF

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select count(*) from usage.usage_basic_kpi_fact_v6_p_{} where date='{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 04, 30)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 05, 23)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(m, day, graularity))
            db_count = result[0][0]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/' \
                      'granularity={}/date={}/'
            v3_count = spark.read.format("delta").load(v3_path.format(graularity, day)).count()

            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print "Completeness Test Pass! date : {}".format(day)
            test_result.append((graularity, day, v3_count, db_count))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'v3_count', 'db_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_v3_db_count_0616/weekly/",
            mode="overwrite",
            partitionBy=["type"])

    retry(write_test_result, (df_write_result,), {}, interval=10)


graularity_list = ["weekly"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select count(*) from usage.usage_basic_kpi_fact_v6_p_{} where date='{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 04, 30)
    start = datetime.date(2013, 01, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 05, 23)
    start = datetime.date(2013, 01, 12)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(m, day, graularity))
            db_count = result[0][0]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/' \
                      'granularity={}/date={}/'
            v3_count = spark.read.format("delta").load(v3_path.format(graularity, day)).count()

            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print "Completeness Test Pass! date : {}".format(day)
            test_result.append((graularity, day, v3_count, db_count))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'v3_count', 'db_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_v3_db_count_0616/monthly/",
            mode="overwrite",
            partitionBy=["type"])

    retry(write_test_result, (df_write_result,), {}, interval=10)


graularity_list = ["monthly"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select count(*) from usage.usage_basic_kpi_fact_v6_p_{} where date='{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2016, 06, 30)
    start = datetime.date(2016, 06, 30)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2016, 06, 25)
    start = datetime.date(2016, 06, 4)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2015, 12, 27)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(m, day, graularity))
            db_count = result[0][0]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v3/fact/' \
                      'granularity={}/date={}/'
            v3_count = spark.read.format("delta").load(v3_path.format(graularity, day)).count()

            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print "Completeness Test Pass! date : {}".format(day)
            test_result.append((graularity, day, v3_count, db_count))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'v3_count', 'db_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_v3_db_count_0616/monthly/",
            mode="append",
            partitionBy=["type"])

    retry(write_test_result, (df_write_result,), {}, interval=10)


graularity_list = ["monthly"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select count(*) from usage.usage_basic_kpi_fact_v6 where date='{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2016, 06, 30)
    start = datetime.date(2016, 06, 30)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2016, 06, 25)
    start = datetime.date(2016, 06, 4)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 06, 14)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(day, graularity))
            db_count = result[0][0]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/' \
                      'granularity={}/date={}/'
            v3_count = spark.read.format("delta").load(v3_path.format(graularity, day)).count()

            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print "Completeness Test Pass! date : {}".format(day)
            test_result.append((graularity, day, v3_count, db_count))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'v3_count', 'db_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_v3_db_count_0616/daily/",
            mode="append",
            partitionBy=["type"])

    retry(write_test_result, (df_write_result,), {}, interval=10)


graularity_list = ["daily"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select count(*) from usage.usage_basic_kpi_fact_v6 where date='{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2016, 06, 30)
    start = datetime.date(2016, 06, 30)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 06, 20)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 06, 14)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(day, graularity))
            db_count = result[0][0]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/' \
                      'granularity={}/date={}/'
            v3_count = spark.read.format("delta").load(v3_path.format(graularity, day)).count()

            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print "Completeness Test Pass! date : {}".format(day)
            test_result.append((graularity, day, v3_count, db_count))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'v3_count', 'db_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_v3_db_count_0616/weekly/",
            mode="append",
            partitionBy=["type"])

    retry(write_test_result, (df_write_result,), {}, interval=10)


graularity_list = ["weekly"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select count(*) from usage.usage_basic_kpi_fact_v6 where date='{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2020, 05, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 06, 20)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 06, 14)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query(aa_dsn, sql.format(day, graularity))
            db_count = result[0][0]

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/' \
                      'granularity={}/date={}/'
            v3_count = spark.read.format("delta").load(v3_path.format(graularity, day)).count()

            if db_count != v3_count:
                print "Completeness Test Fail!!!! date: {}, v3: {}, db: {}".format(day, v3_count, db_count)
            else:
                print v3_count, db_count
                print "Completeness Test Pass! date : {}".format(day)
            test_result.append((graularity, day, v3_count, db_count))
    df_write_result = spark.createDataFrame(test_result, schema=['type', 'date', 'v3_count', 'db_count'])

    from aadatapipelinecore.core.utils.retry import retry

    def write_test_result(df_write_result):
        df_write_result.write.format("delta").save(
            "s3://b2c-prod-data-pipeline-qa/aa.usage/result_usage_v3_db_count_0616/monthly/",
            mode="append",
            partitionBy=["type"])

    retry(write_test_result, (df_write_result,), {}, interval=10)


graularity_list = ["monthly"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=usage;
select count(1) from usage_basic_kpi_fact_v6 where granularity='weekly' and date='2020-06-20';
EOF

In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF
select count(uniqlo_id) from plproxy.execute_select_nestloop(\$proxy\$ 
                    select max(app_id) as uniqlo_id
                from mu.app_monthly
                where 
                date='2020-05-31'
                group by
                app_id,
                store_id,
                device_id
            \$proxy\$) t (uniqlo_id BIGINT);
EOF

In [0]:

df = spark.read.parquet('s3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type=WEEK/date=2020-06-20')
print df.distinct().count()

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from applications.db_check_v1.common.db_check_utils import query_df
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select * from usage.usage_basic_kpi_fact_v6 where date='{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2020, 05, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 06, 20)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 06, 14)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query_df(aa_dsn, sql.format(day, graularity))

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/' \
                      'granularity={}/date={}/'
            unified_v3 = spark.read.format("delta").load(v3_path.format(graularity, day)).drop('_identifier')
            unified_v3 = unified_v3.drop('date').drop('granularity')
            unified_v3 = unified_v3.na.fill(0)
            
            citus_db_df = spark.createDataFrame(result)
            citus_db_df = citus_db_df.drop('date').drop('granularity').drop('_disable_idx_4_query')

            diff_count1 = citus_db_df.select(unified_v3.columns).subtract(unified_v3).count()
            diff_count2 = unified_v3.select(citus_db_df.columns).subtract(citus_db_df).count()
            print diff_count1, diff_count2


graularity_list = ["daily"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
from pyspark.sql import Row
from applications.db_check_v1.common.db_check_utils import query_df
from dateutil.relativedelta import relativedelta
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime as d
import datetime
from datetime import timedelta


test_result = []
PG_AA_HOSTS = [('10.2.6.141', 5432)]
PG_AA_NAME = 'aa_store_db'
PG_AA_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
PG_AA_SECRET_KEY = 'wZw8cfBuuklIskVG'

aa_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=PG_AA_NAME,
        user=PG_AA_ACCESS_ID,
        host=PG_AA_HOSTS[0][0],
        password=PG_AA_SECRET_KEY,
        port=PG_AA_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
sql = """select * from usage.usage_basic_kpi_fact_v6 where date='{}' and granularity='{}';"""


def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result


def last_day_of_month(check_month):
    next_month = check_month.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def get_monthly_date_list():
    result = []
    end = datetime.date(2020, 05, 31)
    start = datetime.date(2020, 05, 31)
    while start <= end:
        start = last_day_of_month(start)
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(months=1)
    return result


def get_weekly_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 06, 20)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(weeks=1)
    return result


def get_daily_date_list():
    result = []
    end = datetime.date(2020, 06, 20)
    start = datetime.date(2020, 06, 14)
    while start <= end:
        month_data_raw = datetime.datetime.strftime(start, '%Y-%m-%d')
        result.append(Row(month_data_raw))
        start += relativedelta(days=1)
    return result


def get_path_date_list(gran):
    date_list = {}
    if gran == 'daily':
        collect_date = get_daily_date_list()
    if gran == 'weekly':
        collect_date = get_weekly_date_list()
    if gran == 'monthly':
        collect_date = get_monthly_date_list()
    for x in collect_date:
        if x[0][:7] in date_list:
            date_list[x[0][:7]].append(x[0])
        else:
            date_list[x[0][:7]] = [x[0]]
    date_list = sorted(date_list.items(), key=lambda d: datetime.datetime.strptime(d[0] + str(-1), '%Y-%m-%d'),
                       reverse=False)
    return date_list


def check_v3_db_completeness(date_list, graularity):
    """
        date_list:
                [(month,[day1,day2,day3])]
        sample:
            [('2015-12', ['2015-12-27', '2015-12-28', '2015-12-29', '2015-12-30', '2015-12-31'])]
    """
    for month in date_list:
        m = month[0][:4] + month[0][5:8]
        for day in month[1]:
            result = query_df(aa_dsn, sql.format(day, graularity))

            v3_path = 's3://b2c-prod-data-pipeline-unified-usage/unified/usage.basic-kpi.v5/fact/' \
                      'granularity={}/date={}/'
            unified_v3 = spark.read.format("delta").load(v3_path.format(graularity, day)).drop('_identifier')
            unified_v3 = unified_v3.drop('date').drop('granularity')
            unified_v3 = unified_v3.na.fill(0)
            
            citus_db_df = spark.createDataFrame(result)
            citus_db_df = citus_db_df.drop('date').drop('granularity').drop('_disable_idx_4_query')

            diff_count1 = citus_db_df.select(unified_v3.columns).subtract(unified_v3).count()
            diff_count2 = unified_v3.select(citus_db_df.columns).subtract(citus_db_df).count()
            print diff_count1, diff_count2


graularity_list = ["weekly"]
for graularity in graularity_list:
    check_v3_db_completeness(get_path_date_list(graularity), graularity)

In [0]:

from bdce.common.utils import update_application_code
update_application_code(spark, role="BDP-PROD-APP-INT-QA", application_name="zidong-application-autopipeline")

# reload dependencies from temp
spark.sparkContext.addPyFile("/tmp/zeppelin_application_code/libs/python/dependencies.zip")
# spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy

In [0]:

import random
import datetime
from applications.db_check_v1.common.db_check_utils import _get_date_from_refresh_routing_config, etl_skip
from applications.db_check_v1.common.constants import query
from applications.db_check_v1.common.db_check_utils import query_df
from applications.db_check_v1.common.base_test import PipelineTest
from applications.db_check_v1.common.utils import string_to_datetime, datetime_to_string
from pyspark.sql.types import DoubleType
from pyspark.sql import functions
from pyspark.sql.functions import lit, coalesce
from pyspark.sql.types import LongType
CITUS_USAGE_HOSTS = [('10.2.10.254', 5432)]
CITUS_USAGE_NAME = 'aa_store_db'
CITUS_USAGE_ACCESS_ID = 'citus_bdp_prod_app_int_qa'
CITUS_USAGE_SECRET_KEY = 'wZw8cfBuuklIskVG'

DEVICE_CODE_MAPPING = {
    1: {'1': 'android-phone', '2': 'android-tablet'},
    2: {'1': 'ios-phone', '2': 'ios-tablet'}}

GRANULARITY_IN_RAW_PATH_MAPPING = {
    "daily": "DAY",
    "weekly": "WEEK",
    "monthly": "MONTH"
}

CITUS_DSN = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_USAGE_NAME,
        user=CITUS_USAGE_ACCESS_ID,
        host=CITUS_USAGE_HOSTS[0][0],
        password=CITUS_USAGE_SECRET_KEY,
        port=CITUS_USAGE_HOSTS[0][1]
    )
)


class UsageRoutineRawData(object):
    """
    Get data from Data Foundation
    """
    _raw_s3_path = 's3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type={granularity}/date={date}'

    def __init__(self, spark):
        self.spark = spark

    def get(self, granularity, date):
        """
        :return: Raw data from DF team
        :rtype: pyspark.sql.DataFrame
        """
        raw_data = self.spark.read.parquet(self._raw_s3_path.format(
            granularity=GRANULARITY_IN_RAW_PATH_MAPPING[granularity], date=date))
        return raw_data


class TestUsageRoutineRawCompleteness(PipelineTest):
    table_name = 'usage.usage_basic_kpi_fact_v6'
    db_name = 'usage'

    def setUp(self):
        # super(PipelineTest, self).setUp()
        self.check_date = None
        self.granularity = None

    def check_routine_raw_completeness(self):
        date_list = [self.check_date]

        if self.granularity == 'daily':
            date = string_to_datetime(self.check_date)
            weekly_day_nums = 7
            date_list = [datetime_to_string(date - datetime.timedelta(days=x)) for x in range(weekly_day_nums)]

        for date in date_list:
            routine_df = UsageRoutineRawData(self.spark).get(self.granularity, date)
            routine_count = routine_df.count()

            citus_db_count = self.get_citus_db_count(date)
            print routine_count, citus_db_count

            self.assertEqual(routine_count, citus_db_count[0][0],
                             msg="fount count mismatch when compare usage routine raw and citus db. "
                                 "granularity is {}, date is {}, raw count is:{}, citus db count is:{}".format(
                                    self.granularity, date, routine_count, citus_db_count[0][0]))

    def get_citus_db_count(self, date):
        sql = """select count(1) as cnt from {table_name} where date='{date}' and granularity='{granularity}';
        """.format(table_name=self.table_name, date=date, granularity=self.granularity)
        result = query(CITUS_DSN, sql)
        return result


class TestUsageRoutineRawCompletenessDaily(TestUsageRoutineRawCompleteness):

    trigger_date_config = ("0 12 * * 5", 6)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "daily"

    def test_routine_raw_completeness_daily(self):
        self.check_routine_raw_completeness()


class TestUsageRoutineRawCompletenessWeekly(TestUsageRoutineRawCompleteness):

    trigger_date_config = ("0 12 * * 5", 6)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "weekly"

    @etl_skip()
    def test_routine_raw_completeness_weekly(self):
        self.check_routine_raw_completeness()


class TestUsageRoutineRawCompletenessMonthly(TestUsageRoutineRawCompleteness):

    trigger_date_config = ("0 12 6 * * ", 6)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "monthly"

    @etl_skip()
    def test_routine_raw_completeness_monthly(self):
        self.check_routine_raw_completeness()


class TestUsageRoutineRawAccuracy(PipelineTest):
    table_name = 'usage.usage_basic_kpi_fact_v6'
    raw_path = 's3://aardvark-prod-pdx-mdm-to-int/basic_kpi/version=1.0.0/range_type={granularity}/date={date}'
    db_name = 'usage'

    def setUp(self):
        super(PipelineTest, self).setUp()
        self.check_date = None
        self.granularity = None

    def check_routine_raw_accuracy(self):
        routine_df = UsageRoutineRawData(self.spark).get(self.granularity, self.check_date)
        unified_v1 = (
            routine_df
            .withColumn('device_code', functions.UserDefinedFunction(
                lambda x, y: DEVICE_CODE_MAPPING[x][y])(routine_df['platform'], routine_df['device_type']))
            .withColumnRenamed('country', 'country_code')
            .withColumn('app_id', routine_df['app_id'].cast(LongType()))
            .withColumnRenamed('AU', 'est_average_active_users')
            .withColumnRenamed('AFU', 'est_average_session_per_user')
            .withColumnRenamed('ADU', 'est_average_session_duration')
            .withColumnRenamed('IP', 'est_install_penetration')
            .withColumnRenamed('AAD', 'est_average_active_days')
            .withColumnRenamed('PAD', 'est_percentage_active_days')
            .withColumnRenamed('MBPU', 'est_average_bytes_per_user')
            .withColumnRenamed('ATU', 'est_average_time_per_user')
            .withColumnRenamed('UP', 'est_usage_penetration')
            .withColumnRenamed('OR', 'est_open_rate')
            .withColumnRenamed('MBPS', 'est_average_bytes_per_session')
            .withColumnRenamed('MBWFT', 'est_percent_of_wifi_total')
            .withColumnRenamed('MBS', 'est_mb_per_second')
            .withColumnRenamed('IS', 'est_installs')
            .withColumnRenamed('SOU', 'est_average_active_users_country_share')
            .withColumnRenamed('SOI', 'est_installs_country_share')
            .withColumn('est_share_of_category_time', lit(None).cast(DoubleType()))
            .withColumn('est_share_of_category_session', lit(None).cast(DoubleType()))
            .withColumn('est_share_of_category_bytes', lit(None).cast(DoubleType()))
            .withColumn('est_panel_size', lit(None).cast(DoubleType()))
            .drop('device_type')
            .drop('platform')
        )

        unified_v1 = (
            unified_v1
            .drop("est_mb_per_second", "est_share_of_device_time", "est_share_of_device_session",
                  "est_share_of_device_mb", "est_panel_size")
            .withColumn('est_total_time',
                        unified_v1['est_average_active_users'] * unified_v1['est_average_time_per_user'] / 60)
            .withColumn('est_average_time_per_user', coalesce(unified_v1['est_average_time_per_user'],
                                                              unified_v1['est_average_session_duration'] *
                                                              unified_v1['est_average_session_per_user']) * 1000)
            .withColumn('est_average_session_duration', unified_v1['est_average_session_duration'] * 1000)
        )
        unified_v1 = (
            unified_v1
            .withColumn('est_average_bytes_per_session', unified_v1['est_average_bytes_per_session'] * 1024 * 1024)
            .withColumn('est_average_bytes_per_user', unified_v1['est_average_bytes_per_user'] * 1024 * 1024)
            .withColumn('est_share_of_category_bytes', unified_v1['est_share_of_category_bytes'] * 1024 * 1024)
            .withColumn('est_install_base',
                        unified_v1['est_install_penetration'] *
                        unified_v1['est_average_active_users'] / unified_v1['est_usage_penetration'])
            .withColumn('est_population',
                        unified_v1['est_average_active_users'] / unified_v1['est_usage_penetration'])
            .withColumnRenamed('est_average_active_users_country_share', 'est_share_of_users')
            .withColumnRenamed('est_installs_country_share', 'est_share_of_installs')
            .withColumn('est_total_sessions',
                        unified_v1['est_average_active_users'] * unified_v1['est_average_session_per_user'])
        )
        unified_v1.createOrReplaceTempView("v1_df")
        unified_v3 = self.spark.sql("""
                    select
                        v1_df.*,
                        ww.est_average_active_users as est_average_active_users_worldwide,
                        ww.est_installs as est_installs_worldwide
                    from v1_df left join
                        (select device_code, app_id, est_installs, est_average_active_users
                         from v1_df where country_code ='WW'
                         ) AS ww
                    on (v1_df.app_id=ww.app_id) and (v1_df.device_code = ww.device_code)
                    """)
        unified_v3 = unified_v3.na.fill(0)

        result = self.get_citus_db_df()
        citus_db_df = self.spark.createDataFrame(result)
        citus_db_df = citus_db_df.drop('date').drop('granularity').drop('_disable_idx_4_query')

        diff_count1 = citus_db_df.select(unified_v3.columns).subtract(unified_v3).count()
        diff_count2 = unified_v3.select(citus_db_df.columns).subtract(citus_db_df).count()
        self.assertTrue(diff_count1 == 0 and diff_count2 == 0,
                        msg="fount mismatch when compare usage routine raw and citus db.granularity is {}, "
                            "date is {}, diff_count1 is:{}, diff_count2 is:{}".format(
                                self.granularity, self.check_date, diff_count1, diff_count2))

    def get_citus_db_df(self):
        sql = """select * from {table_name} where date='{date}' and granularity='{granularity}';
        """.format(table_name=self.table_name, date=self.check_date, granularity=self.granularity)
        result = query_df(CITUS_DSN, sql)
        return result


class TestUsageRoutineRawAccuracyDaily(TestUsageRoutineRawAccuracy):

    trigger_date_config = ("0 12 * * 5", random.randint(6, 12))
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "daily"

    @etl_skip()
    def test_routine_raw_accuracy_daily(self):
        self.check_routine_raw_accuracy()


class TestUsageRoutineRawAccuracyWeekly(TestUsageRoutineRawAccuracy):
    trigger_date_config = ("0 12 * * 5", 6)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "weekly"

    @etl_skip()
    def test_routine_raw_accuracy_weekly(self):
        self.check_routine_raw_accuracy()


class TestUsageRoutineRawAccuracyMonthly(TestUsageRoutineRawAccuracy):
    trigger_date_config = ("0 12 6 * * ", 6)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "monthly"

    @etl_skip()
    def test_routine_raw_accuracy_monthly(self):
        self.check_routine_raw_accuracy()


suite = unittest.TestSuite()
suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawCompletenessDaily))
# suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawCompletenessWeekly))
# suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawCompletenessMonthly))
# suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawAccuracyDaily))
# suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawAccuracyWeekly))
# suite.addTests(unittest.TestLoader().loadTestsFromTestCase(TestUsageRoutineRawAccuracyMonthly))
runner = unittest.TextTestRunner()
runner.run(suite)